# Titanic Survival Prediction - EDA & Helios ML Framework Results

## Overview
This notebook provides comprehensive exploratory data analysis (EDA) of the Titanic dataset and documents the results from the Helios ML Framework implementation.

**Framework Specifications:**
- ISR-governed (T ≥ 1.5)
- QMV-monitored (C < 0.03)
- RLAD feature engineering
- MoT ensemble voting (5 models)

**Final CV Accuracy:** 83.39% (LightGBM best model)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add src to path
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")

## 1. Load Data

In [ ]:
# Load datasets
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/submission.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Submission shape: {submission.shape}")

# Display first rows
train.head()

## 2. Data Overview & Missing Values

In [ ]:
# Dataset info
print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
print(f"\nTotal passengers in training: {len(train)}")
print(f"Total passengers in test: {len(test)}")
print(f"\nSurvival Rate: {train['Survived'].mean():.2%}")
print(f"Death Rate: {(1 - train['Survived'].mean()):.2%}")

# Missing values
print("\n" + "=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)
print("\nTraining Set:")
missing_train = train.isnull().sum()
missing_train = missing_train[missing_train > 0].sort_values(ascending=False)
missing_pct_train = (missing_train / len(train) * 100).round(2)
missing_df_train = pd.DataFrame({
    'Missing Count': missing_train,
    'Percentage': missing_pct_train
})
print(missing_df_train)

print("\nTest Set:")
missing_test = test.isnull().sum()
missing_test = missing_test[missing_test > 0].sort_values(ascending=False)
missing_pct_test = (missing_test / len(test) * 100).round(2)
missing_df_test = pd.DataFrame({
    'Missing Count': missing_test,
    'Percentage': missing_pct_test
})
print(missing_df_test)

## 3. Target Variable Distribution

In [ ]:
# Survival distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
survived_counts = train['Survived'].value_counts()
axes[0].bar(['Died', 'Survived'], survived_counts.values, color=['#d62728', '#2ca02c'])
axes[0].set_ylabel('Count')
axes[0].set_title('Survival Distribution (Count)')
for i, v in enumerate(survived_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Pie chart
axes[1].pie(survived_counts.values, labels=['Died (0)', 'Survived (1)'], 
            autopct='%1.1f%%', colors=['#d62728', '#2ca02c'], startangle=90)
axes[1].set_title('Survival Distribution (Percentage)')

plt.tight_layout()
plt.show()

print(f"Died: {survived_counts[0]} ({survived_counts[0]/len(train)*100:.1f}%)")
print(f"Survived: {survived_counts[1]} ({survived_counts[1]/len(train)*100:.1f}%)")

## 4. Feature Analysis - Demographics

In [ ]:
# Sex vs Survival
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
sex_survival = train.groupby(['Sex', 'Survived']).size().unstack()
sex_survival.plot(kind='bar', ax=axes[0], color=['#d62728', '#2ca02c'])
axes[0].set_title('Survival by Sex')
axes[0].set_xlabel('Sex')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].legend(['Died', 'Survived'])

# Survival rate
sex_survival_rate = train.groupby('Sex')['Survived'].mean()
sex_survival_rate.plot(kind='bar', ax=axes[1], color=['#1f77b4', '#ff7f0e'])
axes[1].set_title('Survival Rate by Sex')
axes[1].set_xlabel('Sex')
axes[1].set_ylabel('Survival Rate')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("\nSurvival rates by sex:")
for sex in ['male', 'female']:
    rate = train[train['Sex'] == sex]['Survived'].mean()
    print(f"{sex.capitalize()}: {rate:.2%}")

In [ ]:
# Passenger Class vs Survival
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
pclass_survival = train.groupby(['Pclass', 'Survived']).size().unstack()
pclass_survival.plot(kind='bar', ax=axes[0], color=['#d62728', '#2ca02c'])
axes[0].set_title('Survival by Passenger Class')
axes[0].set_xlabel('Passenger Class')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['1st', '2nd', '3rd'], rotation=0)
axes[0].legend(['Died', 'Survived'])

# Survival rate
pclass_survival_rate = train.groupby('Pclass')['Survived'].mean()
pclass_survival_rate.plot(kind='bar', ax=axes[1], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[1].set_title('Survival Rate by Passenger Class')
axes[1].set_xlabel('Passenger Class')
axes[1].set_ylabel('Survival Rate')
axes[1].set_xticklabels(['1st', '2nd', '3rd'], rotation=0)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("\nSurvival rates by passenger class:")
for pclass in [1, 2, 3]:
    rate = train[train['Pclass'] == pclass]['Survived'].mean()
    print(f"Class {pclass}: {rate:.2%}")

In [ ]:
# Age distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Age distribution by survival
train[train['Survived'] == 0]['Age'].dropna().hist(bins=30, ax=axes[0], alpha=0.7, label='Died', color='#d62728')
train[train['Survived'] == 1]['Age'].dropna().hist(bins=30, ax=axes[0], alpha=0.7, label='Survived', color='#2ca02c')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Count')
axes[0].set_title('Age Distribution by Survival')
axes[0].legend()

# Box plot
train.boxplot(column='Age', by='Survived', ax=axes[1])
axes[1].set_xlabel('Survived')
axes[1].set_ylabel('Age')
axes[1].set_title('Age Distribution by Survival (Box Plot)')
axes[1].set_xticklabels(['Died', 'Survived'])
plt.suptitle('')

plt.tight_layout()
plt.show()

print(f"\nMean age of survivors: {train[train['Survived'] == 1]['Age'].mean():.2f}")
print(f"Mean age of non-survivors: {train[train['Survived'] == 0]['Age'].mean():.2f}")

## 5. Feature Analysis - Family & Fare

In [ ]:
# Family size analysis
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train['IsAlone'] = (train['FamilySize'] == 1).astype(int)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Family size vs survival
family_survival_rate = train.groupby('FamilySize')['Survived'].mean()
family_survival_rate.plot(kind='bar', ax=axes[0], color='#1f77b4')
axes[0].set_title('Survival Rate by Family Size')
axes[0].set_xlabel('Family Size')
axes[0].set_ylabel('Survival Rate')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].set_ylim([0, 1])

# Alone vs not alone
alone_survival = train.groupby('IsAlone')['Survived'].mean()
alone_survival.plot(kind='bar', ax=axes[1], color=['#2ca02c', '#d62728'])
axes[1].set_title('Survival Rate: Alone vs With Family')
axes[1].set_xlabel('Traveling Status')
axes[1].set_ylabel('Survival Rate')
axes[1].set_xticklabels(['With Family', 'Alone'], rotation=0)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print(f"\nSurvival rate alone: {alone_survival[1]:.2%}")
print(f"Survival rate with family: {alone_survival[0]:.2%}")

In [ ]:
# Fare analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fare distribution
train[train['Survived'] == 0]['Fare'].dropna().hist(bins=30, ax=axes[0], alpha=0.7, label='Died', color='#d62728')
train[train['Survived'] == 1]['Fare'].dropna().hist(bins=30, ax=axes[0], alpha=0.7, label='Survived', color='#2ca02c')
axes[0].set_xlabel('Fare')
axes[0].set_ylabel('Count')
axes[0].set_title('Fare Distribution by Survival')
axes[0].legend()
axes[0].set_xlim([0, 200])

# Box plot
train.boxplot(column='Fare', by='Survived', ax=axes[1])
axes[1].set_xlabel('Survived')
axes[1].set_ylabel('Fare')
axes[1].set_title('Fare Distribution by Survival (Box Plot)')
axes[1].set_xticklabels(['Died', 'Survived'])
axes[1].set_ylim([0, 300])
plt.suptitle('')

plt.tight_layout()
plt.show()

print(f"\nMedian fare of survivors: ${train[train['Survived'] == 1]['Fare'].median():.2f}")
print(f"Median fare of non-survivors: ${train[train['Survived'] == 0]['Fare'].median():.2f}")

## 6. Helios ML Framework Results

In [ ]:
# Model performance summary
model_results = pd.DataFrame({
    'Model': ['LightGBM', 'XGBoost', 'Random Forest', 'Gradient Boosting', 'Logistic Regression'],
    'Mean CV Accuracy': [0.8339, 0.8327, 0.8271, 0.8249, 0.8025],
    'Std Dev': [0.0139, 0.0202, 0.0174, 0.0217, 0.0164],
    'QMV': [0.0186, 0.0271, 0.0235, 0.0294, 0.0228],
    'QMV Status': ['PASS', 'PASS', 'PASS', 'PASS', 'PASS']
})

print("=" * 70)
print("HELIOS ML FRAMEWORK - MODEL PERFORMANCE SUMMARY")
print("=" * 70)
print(model_results.to_string(index=False))
print("\n" + "=" * 70)
print("QUALITY METRICS")
print("=" * 70)
print(f"ISR Threshold: T ≥ 1.5")
print(f"ISR Value: 0.0304 (FAILED - may require recalibration)")
print(f"\nQMV Threshold: C < 0.03")
print(f"All models: PASSED")
print(f"\nBest Model: LightGBM with 83.39% accuracy")
print(f"Ensemble Method: Weighted voting (MoT)")

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CV Accuracy comparison
axes[0].barh(model_results['Model'], model_results['Mean CV Accuracy'], 
             xerr=model_results['Std Dev'], color='#1f77b4', alpha=0.7)
axes[0].set_xlabel('Mean CV Accuracy')
axes[0].set_title('Model Performance Comparison')
axes[0].set_xlim([0.75, 0.88])
axes[0].axvline(x=0.8339, color='red', linestyle='--', label='Best: 83.39%')
axes[0].legend()

# QMV comparison
colors = ['#2ca02c' if x < 0.03 else '#d62728' for x in model_results['QMV']]
axes[1].barh(model_results['Model'], model_results['QMV'], color=colors, alpha=0.7)
axes[1].set_xlabel('QMV Value')
axes[1].set_title('QMV Monitoring (C < 0.03)')
axes[1].axvline(x=0.03, color='red', linestyle='--', label='Threshold: 0.03')
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. Prediction Analysis

In [ ]:
# Analyze predictions
pred_counts = submission['Survived'].value_counts()

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.bar(['Died (0)', 'Survived (1)'], pred_counts.values, color=['#d62728', '#2ca02c'])
ax.set_ylabel('Count')
ax.set_title('Test Set Predictions Distribution')
for i, v in enumerate(pred_counts.values):
    ax.text(i, v + 5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nTest Set Predictions:")
print(f"Predicted to die: {pred_counts[0]} ({pred_counts[0]/len(submission)*100:.1f}%)")
print(f"Predicted to survive: {pred_counts[1]} ({pred_counts[1]/len(submission)*100:.1f}%)")
print(f"\nComparison with training set:")
print(f"Training survival rate: {train['Survived'].mean():.2%}")
print(f"Test prediction survival rate: {submission['Survived'].mean():.2%}")

## 8. Feature Importance (from Framework)

In [ ]:
# Key features used in the model
features = [
    'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
    'Sex_encoded', 'Title_encoded', 'FamilySize', 'IsAlone',
    'Deck_encoded', 'Embarked_encoded', 'FarePerPerson',
    'HasCabin', 'Pclass_Sex', 'Pclass_Age', 'SurnameCount',
    'TicketGroupSize', 'AgeBin', 'IsChild'
]

print("=" * 60)
print("ENGINEERED FEATURES (19 total)")
print("=" * 60)
for i, feature in enumerate(features, 1):
    print(f"{i:2d}. {feature}")

print("\n" + "=" * 60)
print("KEY FEATURE ENGINEERING TECHNIQUES")
print("=" * 60)
print("1. Title Extraction: Mr., Mrs., Miss., Master., Rare")
print("2. Family Features: FamilySize, IsAlone, SurnameCount")
print("3. Fare Engineering: FarePerPerson")
print("4. Age Engineering: Age imputation, AgeBin, IsChild")
print("5. Cabin Engineering: Deck extraction, HasCabin")
print("6. Interaction Features: Pclass_Sex, Pclass_Age")
print("7. Ticket Groups: TicketGroupSize")


## 9. Conclusions

### Key Findings:
1. **Gender Impact**: Women had 74% survival rate vs 19% for men
2. **Class Impact**: 1st class: 63%, 2nd class: 47%, 3rd class: 24%
3. **Age Impact**: Children had higher survival rates
4. **Family Impact**: Small families (2-4 members) had better survival rates

### Model Performance:
- **Best Model**: LightGBM with 83.39% CV accuracy
- **Ensemble**: Weighted voting across 5 models
- **QMV Compliance**: All models passed C < 0.03 threshold
- **Features**: 19 engineered features with RLAD abstractions

### Recommendations:
1. ISR metric may need recalibration for small datasets
2. Consider additional feature engineering (ticket prefix analysis)
3. Hyperparameter tuning could improve performance further
4. Ensemble weights could be optimized with grid search